In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
from epiweeks import Week
from datetime import date, datetime
from os.path import dirname, join, isfile, isdir

from bulletin.commom import static
from bulletin.commom.normalize import normalize_text, normalize_number, normalize_hash, normalize_cpf, normalize_igbe
from bulletin.commom.utils import isvaliddate

from bulletin.commom.utils import Timer, auto_fit_columns

In [2]:
municipios = static.municipios.loc[static.municipios['uf']=='PR',['ibge','municipio','populacao']].copy()
municipios['ibge'] = municipios['ibge'].apply(str)

regionais = static.regionais[['ibge','nm_macro','nu_reg','nm_reg']].copy()
regionais['ibge'] = regionais['ibge'].apply(str)
regionais = regionais.rename(columns={'nu_reg':'rs'})

populacao_macro = pd.merge(regionais,municipios,on='ibge',how='inner').groupby('nm_macro')[['populacao']].sum()
populacao_rs = pd.merge(regionais,municipios,on='ibge',how='inner').groupby('rs')[['populacao']].sum()
populacao_mun = municipios

In [3]:
casos = pd.read_excel('input/casoseobitos.xlsx','casos',converters={
        'ibge': normalize_igbe
    }
)

In [4]:
casos = pd.merge(left=casos, right=regionais, how='left', on='ibge')
casos = pd.merge(left=casos, right=municipios, how='left', on='ibge')
casos['ano'] = casos.apply(lambda row: Week.fromdate(row['dt_diag']).year, axis=1)
casos['mes'] = casos.apply(lambda row: Week.fromdate(row['dt_diag']).startdate().month, axis=1)
casos['se'] = casos.apply(lambda row: Week.fromdate(row['dt_diag']).week, axis=1)

In [5]:
# casos_macro = casos.groupby(['ano','mes','se','nm_macro'])[['dt_diag']].count().rename(columns={'dt_diag':'casos'})
# casos_macro = casos_macro.unstack(level=[0,1,2]).swaplevel(0, 1, axis=1).swaplevel(1, 2, axis=1).swaplevel(2, 3, axis=1).fillna(0).astype(int)
# casos_macro = casos_macro.xs('casos', axis=1, level= 3)
# casos_macro['total'] = casos_macro.sum(axis=1)

# tuples = [ (macro,populacao) for macro, populacao in zip(casos_macro.index,populacao_macro['populacao'].values) ]
# casos_macro = casos_macro.set_index(pd.MultiIndex.from_tuples(tuples, names=["macro", "populacao"]))

In [6]:
# casos_rs = casos.groupby(['ano','mes','se','nm_macro','rs','nm_reg'])[['dt_diag']].count().rename(columns={'dt_diag':'casos'})

# casos_rs = casos_rs.unstack(level=[0,1,2]).swaplevel(0, 1, axis=1).swaplevel(1, 2, axis=1).swaplevel(2, 3, axis=1).fillna(0).astype(int)
# casos_rs = casos_rs.xs('casos', axis=1, level= 3)
# casos_rs['total'] = casos_rs.sum(axis=1)

# tuples = [ ( macro, rs, nm_rs, populacao) for (macro, rs, nm_rs), populacao in zip(casos_rs.index,populacao_rs['populacao'].values) ]
# casos_rs = casos_rs.set_index(pd.MultiIndex.from_tuples(tuples, names=["macro","rs","nome","populacao"]))

In [16]:
# casos_mun = casos.groupby(['ano','mes','se','nm_macro','rs','nm_reg','ibge','municipio','populacao'])[['dt_diag']].count().rename(columns={'dt_diag':'casos'})

# casos_mun = casos_mun.unstack(level=[0,1,2]).swaplevel(0, 1, axis=1).swaplevel(1, 2, axis=1).swaplevel(2, 3, axis=1).fillna(0).astype(int)
# casos_mun = casos_mun.xs('casos', axis=1, level= 3)
# casos_mun['total'] = casos_mun.sum(axis=1)

ano                                                             2020        \
mes                                                            Abril         
se                                                                15    16   
                                                               casos casos   
nm_macro rs nm_reg     ibge   municipio              populacao               
LESTE    1   PARANAGUA 410120 Antonina               18949         0     0   
                       410950 Guaraqueçaba           7594          0     0   
                       410960 Guaratuba              37527         0     0   
                       411570 Matinhos               35219         0     0   
                       411620 Morretes               16446         0     1   
...                                                              ...   ...   
OESTE    20 TOLEDO     412545 São José das Palmeiras 3627          0     0   
                       412575 São Pedro do Iguaçu    5820          0     0   
                       412740 Terra Roxa             17522         0     0   
                       412770 Toledo                 142645        6     1   
                       412795 Tupãssi                8109          0     1   

ano                                                                         \
mes                                                                          
se                                                                17    18   
                                                               casos casos   
nm_macro rs nm_reg     ibge   municipio              populacao               
LESTE    1   PARANAGUA 410120 Antonina               18949         0     0   
                       410950 Guaraqueçaba           7594          0     0   
                       410960 Guaratuba              37527         2     1   
                       411570 Matinhos               35219         0     0   
                       411620 Morretes               16446         0     1   
...                                                              ...   ...   
OESTE    20 TOLEDO     412545 São José das Palmeiras 3627          0     0   
                       412575 São Pedro do Iguaçu    5820          0     0   
                       412740 Terra Roxa             17522         0     0   
                       412770 Toledo                 142645        0     5   
                       412795 Tupãssi                8109          0     0   

ano                                                                         \
mes                                                              Ago         
se                                                                32    33   
                                                               casos casos   
nm_macro rs nm_reg     ibge   municipio              populacao               
LESTE    1   PARANAGUA 410120 Antonina               18949        81    29   
                       410950 Guaraqueçaba           7594          5     9   
                       410960 Guaratuba              37527        71    67   
                       411570 Matinhos               35219        37    25   
                       411620 Morretes               16446        67    35   
...                                                              ...   ...   
OESTE    20 TOLEDO     412545 São José das Palmeiras 3627          5     6   
                       412575 São Pedro do Iguaçu    5820          0     5   
                       412740 Terra Roxa             17522         7    14   
                       412770 Toledo                 142645      145   307   
                       412795 Tupãssi                8109          2     6   

ano                                                                         \
mes                                                                          
se                                                                34    35   
                   

In [8]:
obitos = pd.read_excel('input/casoseobitos.xlsx','obitos',converters={
        'ibge': normalize_igbe
    }
)

In [9]:
obitos = pd.merge(left=obitos, right=regionais, how='left', on='ibge')
obitos = pd.merge(left=obitos, right=municipios, how='left', on='ibge')
obitos['ano'] = obitos.apply(lambda row: Week.fromdate(row['data_obito']).year, axis=1)
obitos['mes'] = obitos.apply(lambda row: Week.fromdate(row['data_obito']).startdate().month, axis=1)
obitos['se'] = obitos.apply(lambda row: Week.fromdate(row['data_obito']).week, axis=1)

In [10]:
# obitos_macro = obitos.groupby(['ano','mes','se','nm_macro'])[['data_obito']].count().rename(columns={'data_obito':'obitos'})
# obitos_macro = obitos_macro.unstack(level=[0,1,2]).swaplevel(0, 1, axis=1).swaplevel(1, 2, axis=1).swaplevel(2, 3, axis=1).fillna(0).astype(int)
# obitos_macro = obitos_macro.xs('obitos', axis=1, level= 3)
# obitos_macro['total'] = obitos_macro.sum(axis=1)

# tuples = [ (macro,populacao) for macro, populacao in zip(obitos_macro.index,populacao_macro['populacao'].values) ]
# obitos_macro = obitos_macro.set_index(pd.MultiIndex.from_tuples(tuples, names=["macro", "populacao"]))

In [11]:
# obitos_rs = obitos.groupby(['ano','mes','se','nm_macro','rs','nm_reg'])[['data_obito']].count().rename(columns={'data_obito':'obitos'})

# obitos_rs = obitos_rs.unstack(level=[0,1,2]).swaplevel(0, 1, axis=1).swaplevel(1, 2, axis=1).swaplevel(2, 3, axis=1).fillna(0).astype(int)
# obitos_rs = obitos_rs.xs('obitos', axis=1, level= 3)
# obitos_rs['total'] = obitos_rs.sum(axis=1)

# tuples = [ ( macro, rs, nm_rs, populacao) for (macro, rs, nm_rs), populacao in zip(obitos_rs.index,populacao_rs['populacao'].values) ]
# obitos_rs = obitos_rs.set_index(pd.MultiIndex.from_tuples(tuples, names=["macro","rs","nome","populacao"]))

In [12]:
# obitos_mun = obitos.groupby(['ano','mes','se','nm_macro','rs','nm_reg','ibge','municipio','populacao'])[['data_obito']].count().rename(columns={'data_obito':'obitos'})

# obitos_mun = obitos_mun.unstack(level=[0,1,2]).swaplevel(0, 1, axis=1).swaplevel(1, 2, axis=1).swaplevel(2, 3, axis=1).fillna(0).astype(int)
# obitos_mun = obitos_mun.xs('obitos', axis=1, level= 3)
# obitos_mun['total'] = obitos_mun.sum(axis=1)

In [13]:
# sheets = { 'casos_macro': casos_macro, 'casos_rs': casos_rs,'casos_mun': casos_mun, 'obitos_macro': obitos_macro,'obitos_rs': obitos_rs,'obitos_mun': obitos_mun }

# writer = pd.ExcelWriter(join('output','casos_obitos.xlsx'),
#                         engine='xlsxwriter',
#                         datetime_format='dd/mm/yyyy',
#                         date_format='dd/mm/yyyy')


# for sheet_name, sheet in sheets.items():
#     sheet.to_excel(writer,sheet_name)
#     worksheet = writer.sheets[sheet_name]
#     auto_fit_columns(worksheet,sheet)

# writer.save()
# writer.close()

In [34]:
casos.groupby(['ano','mes','se'])[['dt_diag']].count()

dt_diag
ano  mes   se         
2020 Abril 15      440
           16      352
           17      416
           18      541
     Ago   32    16022
           33    15219
           34    14609
           35    14832
           36    15268
     Dez   50    33141
           51    25599
           52    19492
           53    20251
     Jul   28    13887
           29    14827
           30    15560
           31    14628
     Jun   24     4008
           25     6801
           26     8988
           27    13258
     Mai   19      540
           20      787
           21     1381
           22     2168
           23     3047
     Mar   11       12
           12       73
           13      167
           14      439
     Nov   45     9905
           46    17216
           47    23187
           48    27768
           49    30591
     Out   41     8688
           42     5718
           43     7519
           44     8516
     Set   37    13416
           38    13225
           39    11452
           40    11017
2021 Fev   6     17469
           7     20814
           8     13496
     Jan   1     25001
           2     27162
           3     22919
           4     19598
           5     17569

In [14]:
writer = pd.ExcelWriter(join('output','dados_fontes.xlsx'),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

populacao_macro.to_excel(writer,'populacao_macro')
worksheet = writer.sheets['populacao_macro']
auto_fit_columns(worksheet,populacao_macro)

populacao_rs.to_excel(writer,'populacao_rs')
worksheet = writer.sheets['populacao_rs']
auto_fit_columns(worksheet,populacao_rs)

populacao_mun.to_excel(writer,'populacao_mun',index=False)
worksheet = writer.sheets['populacao_mun']
auto_fit_columns(worksheet,populacao_mun)

casos.to_excel(writer,'casos',index=False)
worksheet = writer.sheets['casos']
auto_fit_columns(worksheet,casos)

obitos.to_excel(writer,'obitos',index=False)
worksheet = writer.sheets['obitos']
auto_fit_columns(worksheet,obitos)

writer.save()
writer.close()